# Titanic 生存者予測（Kaggleデータ分析）

このノートブックでは、KaggleのTitanicコンペ用データを使って、
乗客が生存するかどうかを予測するモデルを構築しています。

## 使用技術
- データ前処理（欠損値処理、カテゴリ変換）
- ランダムフォレストによる分類
- 精度: 0.74880（Kaggle Public Score）

## 目的
実践的な前処理とモデル作成の流れを体験し、
分析業務に必要な基礎力を身につける。


In [1]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

In [2]:
import pandas as pd

# 読み込み　
train = pd.read_csv("train.csv")
test  = pd.read_csv("test.csv")

# 中身を見てみよう
train.head()



,PassengerId,Survived,Pclass,Name,Sex,Age,SibSp,Parch,Ticket,Fare,Cabin,Embarked
0,1,0,3,"Braund, Mr. Owen Harris",male,22.0,1,0,A/5 21171,7.2500,NaN,S
1,2,1,1,"Cumings, Mrs. John Bradley (Florence Briggs Th...",female,38.0,1,0,PC 17599,71.2833,C85,C
2,3,1,3,"Heikkinen, Miss. Laina",female,26.0,0,0,STON/O2. 3101282,7.9250,NaN,S
3,4,1,1,"Futrelle, Mrs. Jacques Heath (Lily May Peel)",female,35.0,1,0,113803,53.1000,C123,S
4,5,0,3,"Allen, Mr. William Henry",male,35.0,0,0,373450,8.0500,NaN,S


In [3]:
train.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 891 entries, 0 to 890
Data columns (total 12 columns):
 #   Column       Non-Null Count  Dtype  
---  ------       --------------  -----  
 0   PassengerId  891 non-null    int64  
 1   Survived     891 non-null    int64  
 2   Pclass       891 non-null    int64  
 3   Name         891 non-null    object 
 4   Sex          891 non-null    object 
 5   Age          714 non-null    float64
 6   SibSp        891 non-null    int64  
 7   Parch        891 non-null    int64  
 8   Ticket       891 non-null    object 
 9   Fare         891 non-null    float64
 10  Cabin        204 non-null    object 
 11  Embarked     889 non-null    object 
dtypes: float64(2), int64(5), object(5)
memory usage: 83.7+ KB


In [4]:
train.groupby("Sex")["Survived"].mean()

Sex
female    0.742038
male      0.188908
Name: Survived, dtype: float64

In [5]:
train.isnull().sum()

PassengerId      0
Survived         0
Pclass           0
Name             0
Sex              0
Age            177
SibSp            0
Parch            0
Ticket           0
Fare             0
Cabin          687
Embarked         2
dtype: int64

In [6]:
# Age：中央値で埋める
train["Age"] = train["Age"].fillna(train["Age"].median())
test["Age"] = test["Age"].fillna(test["Age"].median())

# Embarked：最頻値で埋める
train["Embarked"] = train["Embarked"].fillna(train["Embarked"].mode()[0])

# Cabin：削除
train.drop("Cabin", axis=1, inplace=True, errors='ignore')
test.drop("Cabin", axis=1, inplace=True, errors='ignore')



In [7]:
train["Sex"] = train["Sex"].map({"male": 0, "female": 1})
test["Sex"] = test["Sex"].map({"male": 0, "female": 1})

In [8]:
train[["Sex"]].head()

,Sex
0,0
1,1
2,1
3,1
4,0


In [9]:
# ワンホットエンコーディング
train = pd.get_dummies(train, columns=["Embarked"], drop_first=True)
test = pd.get_dummies(test, columns=["Embarked"], drop_first=True)

In [10]:
features = ["Pclass", "Sex", "Age", "SibSp", "Parch", "Fare", "Embarked_Q", "Embarked_S"]


In [11]:
print(train.columns)


Index(['PassengerId', 'Survived', 'Pclass', 'Name', 'Sex', 'Age', 'SibSp',
       'Parch', 'Ticket', 'Fare', 'Embarked_Q', 'Embarked_S'],
      dtype='object')


In [12]:
# もとのtestデータを読み直してリセット！
test = pd.read_csv("test.csv")


In [13]:
# 欠損処理
test["Age"] = test["Age"].fillna(test["Age"].median())
test["Fare"] = test["Fare"].fillna(test["Fare"].median())
test["Embarked"] = test["Embarked"].fillna("S")

# 数値変換
test["Sex"] = test["Sex"].map({"male": 0, "female": 1})

# ワンホットエンコーディング
test = pd.get_dummies(test, columns=["Embarked"], drop_first=True)

# 欠けてる列を補完
for col in ["Embarked_Q", "Embarked_S"]:
    if col not in test.columns:
        test[col] = 0


In [14]:
from sklearn.ensemble import RandomForestClassifier

# モデル定義と再学習
X = train[features]
y = train["Survived"]

model = RandomForestClassifier(n_estimators=100, random_state=42)
model.fit(X, y)


,n_estimators,100
,criterion,'gini'
,max_depth,None
,min_samples_split,2
,min_samples_leaf,1
,min_weight_fraction_leaf,0.0
,max_features,'sqrt'
,max_leaf_nodes,None
,min_impurity_decrease,0.0
,bootstrap,True
,oob_score,False


In [15]:
# Sex を数値に変換（0: male, 1: female）
train["Sex"] = train["Sex"].map({"male": 0, "female": 1})


In [16]:
print(train["Sex"].unique())


[nan]


In [17]:
print(test.columns)


Index(['PassengerId', 'Pclass', 'Name', 'Sex', 'Age', 'SibSp', 'Parch',
       'Ticket', 'Fare', 'Cabin', 'Embarked_Q', 'Embarked_S'],
      dtype='object')


In [18]:
print(test[["Embarked_Q", "Embarked_S"]].head())


   Embarked_Q  Embarked_S
0        True       False
1       False        True
2        True       False
3       False        True
4       False        True


In [19]:
test[features].isnull().sum()

Pclass        0
Sex           0
Age           0
SibSp         0
Parch         0
Fare          0
Embarked_Q    0
Embarked_S    0
dtype: int64

In [20]:
# 欠損処理（Age, Fare）
test["Age"] = test["Age"].fillna(test["Age"].median())
test["Fare"] = test["Fare"].fillna(test["Fare"].median())

# 念のため Sex が文字になってたら再変換
test["Sex"] = test["Sex"].map({"male": 0, "female": 1})

# Embarked が無いときの補完（列がないならスキップしてOK）
# test["Embarked"] = test["Embarked"].fillna("S") ← これは再読み込み時だけ必要

# Embarked ワンホットエンコード後に足りない列がないかチェック
for col in ["Embarked_Q", "Embarked_S"]:
    if col not in test.columns:
        test[col] = 0


In [21]:
test[features].isnull().sum()

Pclass          0
Sex           418
Age             0
SibSp           0
Parch           0
Fare            0
Embarked_Q      0
Embarked_S      0
dtype: int64

In [22]:
# 1. test を再読み込み
test = pd.read_csv("test.csv")

# 2. 欠損埋める
test["Age"] = test["Age"].fillna(test["Age"].median())
test["Fare"] = test["Fare"].fillna(test["Fare"].median())

# 3. 性別を数値に変換
test["Sex"] = test["Sex"].map({"male": 0, "female": 1})

# 4. Embarked を仮で埋める（このデータには基本入ってないけど、念のため）
test["Embarked"] = "S"

# 5. ワンホットエンコーディング
test = pd.get_dummies(test, columns=["Embarked"], drop_first=True)

# 6. 足りない列を補完
for col in ["Embarked_Q", "Embarked_S"]:
    if col not in test.columns:
        test[col] = 0


In [23]:
X_test = test[features]
predictions = model.predict(X_test)

output = pd.DataFrame({
    "PassengerId": test["PassengerId"],
    "Survived": predictions
})
output.to_csv("submission.csv", index=False)


In [24]:
X_test = test[features]
predictions = model.predict(X_test)

output = pd.DataFrame({
    "PassengerId": test["PassengerId"],
    "Survived": predictions
})
output.to_csv("submission.csv", index=False)
